In [1]:
import torch
from utils.aihub_kpop_dataset import KpopImageDatasetwT
from torch.utils.data import DataLoader
from model import PoseHRNet
model = PoseHRNet('w48')
model.load_state_dict(torch.load('posehrnet_w48_256x192.pth', map_location='cpu'))

<All keys matched successfully>

In [2]:
x = torch.randn(1, 3, 256, 192)


In [3]:
y = model(x)

RuntimeError: The size of tensor a (48) must match the size of tensor b (24) at non-singleton dimension 3